# Fake News Detection

In [ ]:
# Download the dataset

!wget -O "liar.zip" "https://www.cs.ucsb.edu/~william/data/liar_dataset.zip"

--2021-05-10 01:13:21--  https://www.cs.ucsb.edu/~william/data/liar_dataset.zip
Resolving www.cs.ucsb.edu (www.cs.ucsb.edu)... 23.185.0.3, 2620:12a:8000::3, 2620:12a:8001::3
Connecting to www.cs.ucsb.edu (www.cs.ucsb.edu)|23.185.0.3|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://sites.cs.ucsb.edu/~william/data/liar_dataset.zip [following]
--2021-05-10 01:13:21--  https://sites.cs.ucsb.edu/~william/data/liar_dataset.zip
Resolving sites.cs.ucsb.edu (sites.cs.ucsb.edu)... 128.111.27.164
Connecting to sites.cs.ucsb.edu (sites.cs.ucsb.edu)|128.111.27.164|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1013571 (990K) [application/zip]
Saving to: ‘liar.zip’

liar.zip            100%[===================>] 989.82K  --.-KB/s    in 0.06s   

2021-05-10 01:13:21 (15.0 MB/s) - ‘liar.zip’ saved [1013571/1013571]



In [ ]:
# Module downloads

!pip install optuna

     |████████████████████████████████| 296kB 8.8MB/s 
     |████████████████████████████████| 1.2MB 9.5MB/s 
     |████████████████████████████████| 81kB 9.8MB/s 
     |████████████████████████████████| 81kB 11.0MB/s 
     |████████████████████████████████| 112kB 29.3MB/s 
     |████████████████████████████████| 143kB 25.1MB/s 
     |████████████████████████████████| 51kB 8.4MB/s 
  Created wheel for alembic: filename=alembic-1.6.2-py2.py3-none-any.whl size=164219 sha256=5ec63f77dea5fd295888bce49e3af95318825b5f372e18cd953e1556c4d16d62
  Stored in directory: /root/.cache/pip/wheels/a5/63/44/db29401e49ef5331c163b591f12a465c40af864bfa888dabd2
  Created wheel for pyperclip: filename=pyperclip-1.8.2-cp37-none-any.whl size=11107 sha256=696e1d923de6c395e77e2f8deb026e05c7d212ea2c2113f19bcdd2cd2803fa5b
  Stored in directory: /root/.cache/pip/wheels/25/af/b8/3407109267803f4015e1ee2ff23be0c8c19ce4008665931ee1
Successfully built alembic pyperclip


In [ ]:
# Required modules

import os
import nltk
import string
import optuna
import numpy as np
import pandas as pd

from zipfile import ZipFile
from matplotlib import pyplot as plt

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

In [ ]:
# Package downloads

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# Initial Configration

%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 7)

In [ ]:
# Extracting the dataset

with ZipFile('liar.zip', 'r') as zf:
    zf.extractall('./')

In [ ]:
# Use this column names

columns = ['ID', 'Label', 'Statement', 'Subject', 'Speaker', 'Speaker Job', 'State info', 'Party Affiliation', 'BT', 'F', 'HT', 'MT', 'Pants on fire', 'Context']

In [ ]:
# Loading the dataset

train = pd.read_csv('train.tsv', sep='\t', header=None, names=columns)
train.head()

,ID,Label,Statement,Subject,Speaker,Speaker Job,State info,Party Affiliation,BT,F,HT,MT,Pants on fire,Context
0,2635.json,false,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer
1,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.
2,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver
3,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release
4,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN


In [ ]:
# Inspecting the data

train.info()
train.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10240 entries, 0 to 10239
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ID                 10240 non-null  object 
 1   Label              10240 non-null  object 
 2   Statement          10240 non-null  object 
 3   Subject            10238 non-null  object 
 4   Speaker            10238 non-null  object 
 5   Speaker Job        7343 non-null   object 
 6   State info         8032 non-null   object 
 7   Party Affiliation  10238 non-null  object 
 8   BT                 10238 non-null  float64
 9   F                  10238 non-null  float64
 10  HT                 10238 non-null  float64
 11  MT                 10238 non-null  float64
 12  Pants on fire      10238 non-null  float64
 13  Context            10138 non-null  object 
dtypes: float64(5), object(9)
memory usage: 1.1+ MB


,BT,F,HT,MT,Pants on fire
count,10238.000000,10238.000000,10238.000000,10238.000000,10238.000000
mean,11.533210,13.286482,17.133718,16.434265,6.201407
std,18.973764,24.112936,35.846511,36.151688,16.128927
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2.000000,2.000000,3.000000,3.000000,1.000000
75%,12.000000,12.000000,13.000000,11.000000,5.000000
max,70.000000,114.000000,160.000000,163.000000,105.000000


In [ ]:
# Loading the validation data

valid = pd.read_csv('valid.tsv', sep='\t', header=None, names=columns)
test = pd.read_csv('test.tsv', sep='\t', header=None, names=columns)

In [67]:
# Important functions

def evaluate_on_data(model, X, y, name, typeof="train"):
    model.fit(X, y)
    y_pred = model.predict(X)

    y_num = (y == 'Real').astype('uint8')
    y_pred_num = (y_pred == 'Real').astype('uint8')

    acc = round(accuracy_score(y_pred_num, y_num)*100, 6)
    recall = round(recall_score(y_pred_num, y_num)*100, 6)
    precision = round(precision_score(y_pred_num, y_num)*100, 6)
    f1 = round(f1_score(y_pred_num, y_num)*100, 6)

    border = "+"+"-"*39+"+"
    title = "|"+"".join(name+"("+typeof+")").center(39)+"|"
    subtitle = "|"+"|".join([' Accuracy', '    F1   ', '  Recall ', 'Precision'])+"|"
    vals = "|"+"|".join(map(str, [acc, f1, recall, precision]))+"|"

    print("\n".join([border,title,border,subtitle,border,vals,border,""]))

    return

## Preprocessing

In [ ]:
# Applying some preprocessing

train['Subject'] = train['Subject'].replace(np.nan, "")
train['Speaker'] = train['Speaker'].replace(np.nan, "")
train['Speaker Job'] = train['Speaker Job'].replace(np.nan, "")
train['State info'] = train['State info'].replace(np.nan, "")
train['Party Affiliation'] = train['Party Affiliation'].replace(np.nan, "")

valid['Subject'] = valid['Subject'].replace(np.nan, "")
valid['Speaker'] = valid['Speaker'].replace(np.nan, "")
valid['Speaker Job'] = valid['Speaker Job'].replace(np.nan, "")
valid['State info'] = valid['State info'].replace(np.nan, "")
valid['Party Affiliation'] = valid['Party Affiliation'].replace(np.nan, "")

test['Subject'] = test['Subject'].replace(np.nan, "")
test['Speaker'] = test['Speaker'].replace(np.nan, "")
test['Speaker Job'] = test['Speaker Job'].replace(np.nan, "")
test['State info'] = test['State info'].replace(np.nan, "")
test['Party Affiliation'] = test['Party Affiliation'].replace(np.nan, "")

In [ ]:
# Important preporcessing functions

def remove_punctuation(text, puncts):
    return "".join([c for c in text if c not in puncts])

def remove_stopwords(text, stopwords):
    tokens = word_tokenize(text)

    return " ".join([tok for tok in tokens if tok not in stopwords])

In [ ]:
stems = PorterStemmer()

MAPPER = {'pants-fire': 'Fake', 'false': 'Fake', 'barely-true': 'Fake', 'half-true': 'Real', 'mostly-true': 'Real', 'true': 'Real'}
english_sw = stopwords.words('english')
punctuations = string.punctuation

In [ ]:
# Preprocessing features

train['preprocessed'] = train['Statement'].apply(lambda x: remove_stopwords(x, english_sw))
train['preprocessed'] = train['preprocessed'].apply(lambda x: remove_punctuation(x, punctuations))
train['preprocessed'] = train['preprocessed'].apply(lambda x: stems.stem(x))

valid['preprocessed'] = valid['Statement'].apply(lambda x: remove_stopwords(x, english_sw))
valid['preprocessed'] = valid['preprocessed'].apply(lambda x: remove_punctuation(x, punctuations))
valid['preprocessed'] = valid['preprocessed'].apply(lambda x: stems.stem(x))

test['preprocessed'] = test['Statement'].apply(lambda x: remove_stopwords(x, english_sw))
test['preprocessed'] = test['preprocessed'].apply(lambda x: remove_punctuation(x, punctuations))
test['preprocessed'] = test['preprocessed'].apply(lambda x: stems.stem(x))

train['Label_mapped'] = train['Label'].apply(lambda x: MAPPER[x])
valid['Label_mapped'] = valid['Label'].apply(lambda x: MAPPER[x])
test['Label_mapped'] = test['Label'].apply(lambda x: MAPPER[x])

In [ ]:
# Separating out features and labels

X_train = train['preprocessed']
y_train = train['Label_mapped']

X_valid = valid['preprocessed']
y_valid = valid['Label_mapped']

X_test = test['preprocessed']
y_test = test['Label_mapped']

## Hyperparameter Tuning

### Count Vectorizer

In [ ]:
# Creating a count vectorizer

count_vec = CountVectorizer()
count_vec.fit(X_train)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [ ]:
# Getting the train and valid vectors

count_train = count_vec.transform(X_train)
count_valid = count_vec.transform(X_valid)
count_test = count_vec.transform(X_test)

### Logistic Regression

In [ ]:
# Parameter grid

def objective(trial):
    penalty = trial.suggest_categorical('penalty', ['l1', 'l2', 'elasticnet'])
    C = trial.suggest_loguniform('C', 10e-6, 1)

    params = {'penalty': penalty, 'C': C}

    clf = LogisticRegression(**params)

    return cross_val_score(clf, count_train, y_train, cv=10, n_jobs=-1).mean()

In [ ]:
# Logistic Regression fitting

study = optuna.create_study(study_name='count_logreg', direction='maximize')
study.optimize(objective, n_trials=10)

[I 2021-05-10 01:13:38,112] A new study created in memory with name: count_logreg
[W 2021-05-10 01:13:39,436] Trial 0 failed, because the objective function returned nan.
[W 2021-05-10 01:13:39,529] Trial 1 failed, because the objective function returned nan.
[W 2021-05-10 01:13:39,609] Trial 2 failed, because the objective function returned nan.
[I 2021-05-10 01:13:42,199] Trial 3 finished with value: 0.55 and parameters: {'penalty': 'none', 'C': 0.730611950296751}. Best is trial 3 with value: 0.55.
[W 2021-05-10 01:13:42,290] Trial 4 failed, because the objective function returned nan.
[W 2021-05-10 01:13:42,376] Trial 5 failed, because the objective function returned nan.
[W 2021-05-10 01:13:42,457] Trial 6 failed, because the objective function returned nan.
[W 2021-05-10 01:13:42,543] Trial 7 failed, because the objective function returned nan.
[W 2021-05-10 01:13:42,629] Trial 8 failed, because the objective function returned nan.
[I 2021-05-10 01:13:45,245] Trial 9 finished with

In [ ]:
# Best trial

logreg_trial = study.best_trial

best_logreg = LogisticRegression(**logreg_trial.params)
best_logreg.fit(count_train, y_train)
(best_logreg.predict(count_train) == y_train).mean(), (best_logreg.predict(count_valid) == y_valid).mean()

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:1505: UserWarning:

Setting penalty='none' will ignore the C and l1_ratio parameters

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression



(0.999609375, 0.5654205607476636)

In [ ]:
# logreg evaluation

evaluate_on_data(LogisticRegression(**logreg_trial.params), count_train, y_train, "Logistic Regression Count")
evaluate_on_data(LogisticRegression(**logreg_trial.params), count_valid, y_valid, "Logistic Regression Count", "valid")
evaluate_on_data(LogisticRegression(**logreg_trial.params), count_test, y_test, "Logistic Regression Count", "test")

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:1505: UserWarning:

Setting penalty='none' will ignore the C and l1_ratio parameters

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:1505: UserWarning:

Setting penalty='none' will ignore the C and l1_ratio parameters



+---------------------------------------+
|    Logistic Regression Count(train)   |
+---------------------------------------+
| Accuracy|    F1   |  Recall |Precision|
+---------------------------------------+
|99.960938|99.965236|99.947862|99.982615|
+---------------------------------------+

+---------------------------------------+
|    Logistic Regression Count(valid)   |
+---------------------------------------+
| Accuracy|    F1   |  Recall |Precision|
+---------------------------------------+
|100.0|100.0|100.0|100.0|
+---------------------------------------+

+---------------------------------------+
|    Logistic Regression Count(test)    |
+---------------------------------------+
| Accuracy|    F1   |  Recall |Precision|
+---------------------------------------+
|100.0|100.0|100.0|100.0|
+---------------------------------------+



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:1505: UserWarning:

Setting penalty='none' will ignore the C and l1_ratio parameters



### Multinomail Naive Bayes

In [ ]:
# Parametr grid

def objective(trial):
    alpha = trial.suggest_int('alpha', 1.0, 8.0)

    clf = MultinomialNB(alpha=alpha)

    return cross_val_score(clf, count_train, y_train, cv=10, n_jobs=-1).mean()

In [ ]:
# Multinomail navie bayes fitting

study = optuna.create_study(study_name='count_nb', direction='maximize')
study.optimize(objective, n_trials=100)

[I 2021-05-10 01:13:47,131] A new study created in memory with name: count_nb
[I 2021-05-10 01:13:47,519] Trial 0 finished with value: 0.60615234375 and parameters: {'alpha': 1}. Best is trial 0 with value: 0.60615234375.
[I 2021-05-10 01:13:47,851] Trial 1 finished with value: 0.60927734375 and parameters: {'alpha': 5}. Best is trial 1 with value: 0.60927734375.
[I 2021-05-10 01:13:48,191] Trial 2 finished with value: 0.60654296875 and parameters: {'alpha': 8}. Best is trial 1 with value: 0.60927734375.
[I 2021-05-10 01:13:48,520] Trial 3 finished with value: 0.6095703125 and parameters: {'alpha': 2}. Best is trial 3 with value: 0.6095703125.
[I 2021-05-10 01:13:48,865] Trial 4 finished with value: 0.6095703125 and parameters: {'alpha': 2}. Best is trial 3 with value: 0.6095703125.
[I 2021-05-10 01:13:49,203] Trial 5 finished with value: 0.6064453125 and parameters: {'alpha': 7}. Best is trial 3 with value: 0.6095703125.
[I 2021-05-10 01:13:49,536] Trial 6 finished with value: 0.60654

In [ ]:
# Best params

nb_trial = study.best_trial

nb_best = MultinomialNB(**nb_trial.params)
nb_best.fit(count_train, y_train)
(nb_best.predict(count_train) == y_train).mean(), (nb_best.predict(count_valid) == y_valid).mean()

(0.77001953125, 0.6043613707165109)

In [ ]:
# nb evaluation

evaluate_on_data(MultinomialNB(**nb_trial.params), count_train, y_train, "Multinomial NB Count")
evaluate_on_data(MultinomialNB(**nb_trial.params), count_valid, y_valid, "Multinomial NB Count", "valid")
evaluate_on_data(MultinomialNB(**nb_trial.params), count_test, y_test, "Multinomial NB Count", "test")

+---------------------------------------+
|      Multinomial NB Count(train)      |
+---------------------------------------+
| Accuracy|    F1   |  Recall |Precision|
+---------------------------------------+
|77.001953|81.548225|74.226216|90.472879|
+---------------------------------------+

+---------------------------------------+
|      Multinomial NB Count(valid)      |
+---------------------------------------+
| Accuracy|    F1   |  Recall |Precision|
+---------------------------------------+
|87.227414|88.335704|84.146341|92.964072|
+---------------------------------------+

+---------------------------------------+
|       Multinomial NB Count(test)      |
+---------------------------------------+
| Accuracy|    F1   |  Recall |Precision|
+---------------------------------------+
|82.004736|86.181818|75.961538|99.579832|
+---------------------------------------+



### Support vector Machines

In [ ]:
# Parameter grid

def objective(trial):
    kernel = trial.suggest_categorical('kernel', ['linear', 'poly', 'rbf', 'sigmoid'])
    if kernel is 'poly':
        degree = trial.suggest_int('degree', 1, 4)
        gamma = trial.suggest_categorical('gamma', ['scale','auto'])
        clf = SVC(kernel=kernel, degree=degree, gamma=gamma)

    if kernel in ['rbf', 'sigmoid']:
        gamma = trial.suggest_categorical('gamma', ['scale','auto'])
        clf = SVC(kernel=kernel, gamma=gamma)

    clf = SVC(kernel=kernel)

    return cross_val_score(clf, count_train, y_train, cv=10, n_jobs=-1).mean()

In [ ]:
# Support Vector Machines fitting

study = optuna.create_study(study_name='count_svc', direction='maximize')
study.optimize(objective, n_trials=10)

[I 2021-05-10 01:14:20,599] A new study created in memory with name: count_svc
[I 2021-05-10 01:16:00,338] Trial 0 finished with value: 0.573046875 and parameters: {'kernel': 'poly', 'degree': 2, 'gamma': 'auto'}. Best is trial 0 with value: 0.573046875.
[I 2021-05-10 01:17:25,902] Trial 1 finished with value: 0.60732421875 and parameters: {'kernel': 'sigmoid', 'gamma': 'scale'}. Best is trial 1 with value: 0.60732421875.
/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning:

A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.

[I 2021-05-10 01:20:47,323] Trial 2 finished with value: 0.570703125 and parameters: {'kernel': 'linear'}. Best is trial 1 with value: 0.60732421875.
[I 2021-05-10 01:22:27,852] Trial 3 finished with value: 0.573046875 and parameters: {'kernel': 'poly', 'degree': 1, 'gamma': 'auto'}. Best is trial 1 with value: 0.60732421875.
[I 2021-05-10 

In [ ]:
# Best params

svc_trial = study.best_trial

svc_best = SVC(**svc_trial.params)
svc_best.fit(count_train, y_train)
(svc_best.predict(count_train) == y_train).mean(), (svc_best.predict(count_valid) == y_valid).mean()

(0.56171875, 0.5202492211838006)

In [ ]:
# svc evaluation

evaluate_on_data(SVC(**svc_trial.params, probability=True), count_train, y_train, "SVC Count")
evaluate_on_data(SVC(**svc_trial.params, probability=True), count_valid, y_valid, "SVC Count", "valid")
evaluate_on_data(SVC(**svc_trial.params, probability=True), count_test, y_test, "SVC Count", "test")

+---------------------------------------+
|            SVC Count(train)           |
+---------------------------------------+
| Accuracy|    F1   |  Recall |Precision|
+---------------------------------------+
|56.171875|71.935968|56.171875|100.0|
+---------------------------------------+

+---------------------------------------+
|            SVC Count(valid)           |
+---------------------------------------+
| Accuracy|    F1   |  Recall |Precision|
+---------------------------------------+
|52.024922|68.442623|52.024922|100.0|
+---------------------------------------+

+---------------------------------------+
|            SVC Count(test)            |
+---------------------------------------+
| Accuracy|    F1   |  Recall |Precision|
+---------------------------------------+
|56.353591|72.084806|56.353591|100.0|
+---------------------------------------+



### Random Forest

In [ ]:
# Parameters grid

def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 100, 1000, 100)
    # criterion = trial.suggest_categorical('criterion', ['gini', 'entropy'])
    max_depth = trial.suggest_categorical('max_depth', [4, 5, 6, None])
    max_features = trial.suggest_categorical('max_features', ['auto', 'sqrt', 'log2', None])

    clf = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, max_features=max_features)

    return cross_val_score(clf, count_train, y_train, cv=10, n_jobs=-1).mean()

In [ ]:
# Random Forest fitting

study = optuna.create_study(study_name='count_rf', direction='maximize')
study.optimize(objective, n_trials=10)

[I 2021-05-10 01:38:10,879] A new study created in memory with name: count_rf
[I 2021-05-10 01:39:04,854] Trial 0 finished with value: 0.56171875 and parameters: {'n_estimators': 900, 'max_depth': 6, 'max_features': 'auto'}. Best is trial 0 with value: 0.56171875.
[I 2021-05-10 01:40:05,358] Trial 1 finished with value: 0.56171875 and parameters: {'n_estimators': 1000, 'max_depth': 6, 'max_features': 'sqrt'}. Best is trial 0 with value: 0.56171875.
[I 2021-05-10 01:40:19,348] Trial 2 finished with value: 0.56171875 and parameters: {'n_estimators': 300, 'max_depth': 4, 'max_features': 'sqrt'}. Best is trial 0 with value: 0.56171875.
[I 2021-05-10 01:40:40,241] Trial 3 finished with value: 0.56171875 and parameters: {'n_estimators': 800, 'max_depth': 4, 'max_features': 'log2'}. Best is trial 0 with value: 0.56171875.
[I 2021-05-10 01:40:48,113] Trial 4 finished with value: 0.56171875 and parameters: {'n_estimators': 300, 'max_depth': 4, 'max_features': 'log2'}. Best is trial 0 with value

In [ ]:
# Best params

rf_trial = study.best_trial

rf_best = RandomForestClassifier(**rf_trial.params)
rf_best.fit(count_train, y_train)
(rf_best.predict(count_train) == y_train).mean(), (rf_best.predict(count_valid) == y_valid).mean()

(0.59853515625, 0.5568535825545171)

In [ ]:
# rf evaluation

evaluate_on_data(RandomForestClassifier(**rf_trial.params), count_train, y_train, "Random Forest Count")
evaluate_on_data(RandomForestClassifier(**rf_trial.params), count_valid, y_valid, "Random Forest Count", "valid")
evaluate_on_data(RandomForestClassifier(**rf_trial.params), count_test, y_test, "Random Forest Count", "test")

+---------------------------------------+
|       Random Forest Count(train)      |
+---------------------------------------+
| Accuracy|    F1   |  Recall |Precision|
+---------------------------------------+
|59.853516|72.70796|58.812158|95.201669|
+---------------------------------------+

+---------------------------------------+
|       Random Forest Count(valid)      |
+---------------------------------------+
| Accuracy|    F1   |  Recall |Precision|
+---------------------------------------+
|65.88785|74.08284|61.252446|93.712575|
+---------------------------------------+

+---------------------------------------+
|       Random Forest Count(test)       |
+---------------------------------------+
| Accuracy|    F1   |  Recall |Precision|
+---------------------------------------+
|63.456985|75.5156|60.662702|100.0|
+---------------------------------------+



### Stochastic Gradient Descent

In [ ]:
# Parameters grid

def objective(trial):
    loss = trial.suggest_categorical('loss', ['log', 'modified_huber', 'perceptron'])
    penalty = trial.suggest_categorical('penalty', ['l2', 'l1', 'elasticnet'])
    max_iter = trial.suggest_int('max_iter', 1000, 2000, 100)
    learning_rate = trial.suggest_categorical('learning_rate', ['constant', 'optimal', 'invscaling', 'adaptive'])
    if learning_rate is not 'optimal':
        eta0 = trial.suggest_loguniform('eta0', 10e-6, 1)
    else:
        eta0 = 0

    clf = SGDClassifier(loss=loss, penalty=penalty, learning_rate=learning_rate, eta0=eta0, max_iter=max_iter)

    return cross_val_score(clf, count_train, y_train, cv=10, n_jobs=-1).mean()

In [ ]:
# SGD fitting

study = optuna.create_study(study_name='count_sgd', direction='maximize')
study.optimize(objective, n_trials=100)

[I 2021-05-10 02:03:46,965] A new study created in memory with name: count_sgd
[I 2021-05-10 02:03:47,234] Trial 0 finished with value: 0.546875 and parameters: {'loss': 'perceptron', 'penalty': 'l2', 'max_iter': 1700, 'learning_rate': 'constant', 'eta0': 0.017522383951114166}. Best is trial 0 with value: 0.546875.
[I 2021-05-10 02:03:48,499] Trial 1 finished with value: 0.566796875 and parameters: {'loss': 'modified_huber', 'penalty': 'l2', 'max_iter': 1800, 'learning_rate': 'adaptive', 'eta0': 0.007076240735436297}. Best is trial 1 with value: 0.566796875.
[I 2021-05-10 02:03:49,081] Trial 2 finished with value: 0.58447265625 and parameters: {'loss': 'log', 'penalty': 'l2', 'max_iter': 1800, 'learning_rate': 'optimal'}. Best is trial 2 with value: 0.58447265625.
[I 2021-05-10 02:03:49,762] Trial 3 finished with value: 0.6078125 and parameters: {'loss': 'log', 'penalty': 'l1', 'max_iter': 1200, 'learning_rate': 'invscaling', 'eta0': 0.36959498697581417}. Best is trial 3 with value: 0.

In [ ]:
# Best trial

sgd_trial = study.best_trial

sgd_best = SGDClassifier(**sgd_trial.params)
sgd_best.fit(count_train, y_train)
(sgd_best.predict(count_train) == y_train).mean(), (sgd_best.predict(count_valid) == y_valid).mean()

(0.72353515625, 0.6253894080996885)

In [ ]:
# sgd evaluation

evaluate_on_data(SGDClassifier(**sgd_trial.params), count_train, y_train, "SGD Count")
evaluate_on_data(SGDClassifier(**sgd_trial.params), count_valid, y_valid, "SGD Count", "valid")
evaluate_on_data(SGDClassifier(**sgd_trial.params), count_test, y_test, "SGD Count", "test")

+---------------------------------------+
|            SGD Count(train)           |
+---------------------------------------+
| Accuracy|    F1   |  Recall |Precision|
+---------------------------------------+
|72.197266|77.053276|71.825695|83.10153|
+---------------------------------------+

+---------------------------------------+
|            SGD Count(valid)           |
+---------------------------------------+
| Accuracy|    F1   |  Recall |Precision|
+---------------------------------------+
|90.88785|91.262136|91.058122|91.467066|
+---------------------------------------+

+---------------------------------------+
|            SGD Count(test)            |
+---------------------------------------+
| Accuracy|    F1   |  Recall |Precision|
+---------------------------------------+
|90.292028|91.739422|88.129032|95.658263|
+---------------------------------------+



## Creating Count Vectorizer piplines

In [ ]:
# logreg pipeline

logreg_pipeline = Pipeline([
     ('count_vec', CountVectorizer()),
     ('logreg', LogisticRegression(**logreg_trial.params))
])

In [ ]:
# nb pipeline

nb_pipeline = Pipeline([
      ('count_vec', CountVectorizer()),
      ('nb', MultinomialNB(**nb_trial.params))
])

In [ ]:
# svc pipeline

svc_pipeline = Pipeline([
      ('count_vec', CountVectorizer()),
      ('svc', SVC(**svc_trial.params, probability=True))
])

In [ ]:
# rf pipeline

rf_pipeline = Pipeline([
      ('count_vec', CountVectorizer()),
      ('rf', RandomForestClassifier(**rf_trial.params))
])

In [ ]:
# sgd pipeline

sgd_pipeline = Pipeline([
      ('count_vec', CountVectorizer()),
      ('pa', SGDClassifier(**sgd_trial.params))
])

In [ ]:
# Ensemble of the above classifiers

list_clfs = [('logreg_pipeline', logreg_pipeline), ('nb_pipeline', nb_pipeline), ('svc_pipeline', svc_pipeline), ('rf_pipeline', rf_pipeline), ('sgd_pipeline', sgd_pipeline)]

count_detector = VotingClassifier(estimators=list_clfs, voting='soft', n_jobs=-1)
count_detector.fit(X_train, y_train)

VotingClassifier(estimators=[('logreg_pipeline',
                              Pipeline(memory=None,
                                       steps=[('count_vec',
                                               CountVectorizer(analyzer='word',
                                                               binary=False,
                                                               decode_error='strict',
                                                               dtype=<class 'numpy.int64'>,
                                                               encoding='utf-8',
                                                               input='content',
                                                               lowercase=True,
                                                               max_df=1.0,
                                                               max_features=None,
                                                               min_df=1,
                                  

In [ ]:
# Prediction from voting classifier

print((count_detector.predict(X_valid) == y_valid).mean())
evaluate_on_data(count_detector, X_train, y_train, "Ensemble Classifier Count")
evaluate_on_data(count_detector, X_valid, y_valid, "Ensemble Classifier Count", "valid")
evaluate_on_data(count_detector, X_test, y_test, "Ensemble Classifier Count", "test")

0.5763239875389408
+---------------------------------------+
|    Ensemble Classifier Count(train)   |
+---------------------------------------+
| Accuracy|    F1   |  Recall |Precision|
+---------------------------------------+
|90.888672|92.34053|87.478613|97.774687|
+---------------------------------------+

[[3683  805]
 [ 128 5624]]
+---------------------------------------+
|    Ensemble Classifier Count(valid)   |
+---------------------------------------+
| Accuracy|    F1   |  Recall |Precision|
+---------------------------------------+
|97.507788|97.608371|97.462687|97.754491|
+---------------------------------------+

[[599  17]
 [ 15 653]]
+---------------------------------------+
|    Ensemble Classifier Count(test)    |
+---------------------------------------+
| Accuracy|    F1   |  Recall |Precision|
+---------------------------------------+
|97.632202|97.942387|95.967742|100.0|
+---------------------------------------+

[[523  30]
 [  0 714]]


## Tfidf Vectorizer

In [62]:
# Defining tfidf vector

tfidf = TfidfVectorizer()
tfidf.fit(X_train, y_train)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [63]:
# Getting vectors of train and valid

tfidf_train = tfidf.transform(X_train)
tfidf_valid = tfidf.transform(X_valid)
tfidf_test = tfidf.transform(X_test)

## Hyperparameter tuning (Tfidf)

### Logistic Regression

In [68]:
# Parameter grid

def objective(trial):
    penalty = trial.suggest_categorical('penalty', ['l1', 'l2', 'elasticnet'])
    C = trial.suggest_loguniform('C', 10e-6, 1)

    params = {'penalty': penalty, 'C': C}

    clf = LogisticRegression(**params)

    return cross_val_score(clf, tfidf_train, y_train, cv=10, n_jobs=-1).mean()

In [69]:
# Logistic Regression fitting

study = optuna.create_study(study_name='tfidf_logreg', direction='maximize')
study.optimize(objective, n_trials=10)

[I 2021-05-10 02:36:59,940] A new study created in memory with name: tfidf_logreg
[W 2021-05-10 02:37:00,040] Trial 0 failed, because the objective function returned nan.
[W 2021-05-10 02:37:00,131] Trial 1 failed, because the objective function returned nan.
[W 2021-05-10 02:37:00,224] Trial 2 failed, because the objective function returned nan.
[I 2021-05-10 02:37:00,881] Trial 3 finished with value: 0.56318359375 and parameters: {'penalty': 'l2', 'C': 0.032013841632899935}. Best is trial 3 with value: 0.56318359375.
[I 2021-05-10 02:37:01,373] Trial 4 finished with value: 0.56171875 and parameters: {'penalty': 'l2', 'C': 2.3505942936436858e-05}. Best is trial 3 with value: 0.56318359375.
[W 2021-05-10 02:37:01,471] Trial 5 failed, because the objective function returned nan.
[I 2021-05-10 02:37:01,998] Trial 6 finished with value: 0.56171875 and parameters: {'penalty': 'l2', 'C': 0.005880592095153536}. Best is trial 3 with value: 0.56318359375.
[I 2021-05-10 02:37:02,775] Trial 7 fi

In [70]:
# Best trial

logreg_trial = study.best_trial

best_logreg = LogisticRegression(**logreg_trial.params)
best_logreg.fit(tfidf_train, y_train)
(best_logreg.predict(tfidf_train) == y_train).mean(), (best_logreg.predict(tfidf_valid) == y_valid).mean()

(0.7693359375, 0.6238317757009346)

In [71]:
# logreg evaluation

evaluate_on_data(LogisticRegression(**logreg_trial.params), tfidf_train, y_train, "Logistic Regression Tfidf")
evaluate_on_data(LogisticRegression(**logreg_trial.params), tfidf_valid, y_valid, "Logistic Regression Tfidf", "valid")
evaluate_on_data(LogisticRegression(**logreg_trial.params), tfidf_test, y_test, "Logistic Regression Tfidf", "test")

+---------------------------------------+
|    Logistic Regression Tfidf(train)   |
+---------------------------------------+
| Accuracy|    F1   |  Recall |Precision|
+---------------------------------------+
|76.933594|81.319203|74.593732|89.377608|
+---------------------------------------+

+---------------------------------------+
|    Logistic Regression Tfidf(valid)   |
+---------------------------------------+
| Accuracy|    F1   |  Recall |Precision|
+---------------------------------------+
|90.264798|90.909091|88.401697|93.562874|
+---------------------------------------+

+---------------------------------------+
|    Logistic Regression Tfidf(test)    |
+---------------------------------------+
| Accuracy|    F1   |  Recall |Precision|
+---------------------------------------+
|85.003946|88.139825|79.504505|98.879552|
+---------------------------------------+



### Multinomial Naive Bayes

In [72]:
# Parametr grid

def objective(trial):
    alpha = trial.suggest_int('alpha', 1.0, 8.0)

    clf = MultinomialNB(alpha=alpha)

    return cross_val_score(clf, tfidf_train, y_train, cv=10, n_jobs=-1).mean()

In [73]:
# Multinomail navie bayes fitting

study = optuna.create_study(study_name='tfidf_nb', direction='maximize')
study.optimize(objective, n_trials=100)

[I 2021-05-10 02:37:21,513] A new study created in memory with name: tfidf_nb
[I 2021-05-10 02:37:21,870] Trial 0 finished with value: 0.56943359375 and parameters: {'alpha': 8}. Best is trial 0 with value: 0.56943359375.
[I 2021-05-10 02:37:22,229] Trial 1 finished with value: 0.59765625 and parameters: {'alpha': 2}. Best is trial 1 with value: 0.59765625.
[I 2021-05-10 02:37:22,579] Trial 2 finished with value: 0.5736328125 and parameters: {'alpha': 6}. Best is trial 1 with value: 0.59765625.
[I 2021-05-10 02:37:22,930] Trial 3 finished with value: 0.5736328125 and parameters: {'alpha': 6}. Best is trial 1 with value: 0.59765625.
[I 2021-05-10 02:37:23,280] Trial 4 finished with value: 0.5908203125 and parameters: {'alpha': 3}. Best is trial 1 with value: 0.59765625.
[I 2021-05-10 02:37:23,626] Trial 5 finished with value: 0.5765625 and parameters: {'alpha': 5}. Best is trial 1 with value: 0.59765625.
[I 2021-05-10 02:37:23,984] Trial 6 finished with value: 0.59765625 and parameters:

In [74]:
# Best params

nb_trial = study.best_trial

nb_best = MultinomialNB(**nb_trial.params)
nb_best.fit(tfidf_train, y_train)
(nb_best.predict(tfidf_train) == y_train).mean(), (nb_best.predict(tfidf_valid) == y_valid).mean()

(0.78232421875, 0.5965732087227414)

In [75]:
# nb evaluation

evaluate_on_data(MultinomialNB(**nb_trial.params), tfidf_train, y_train, "Multinomial NB Tfidf")
evaluate_on_data(MultinomialNB(**nb_trial.params), tfidf_valid, y_valid, "Multinomial NB Tfidf", "valid")
evaluate_on_data(MultinomialNB(**nb_trial.params), tfidf_test, y_test, "Multinomial NB Tfidf", "test")

+---------------------------------------+
|      Multinomial NB Tfidf(train)      |
+---------------------------------------+
| Accuracy|    F1   |  Recall |Precision|
+---------------------------------------+
|78.232422|83.203979|73.427317|95.984006|
+---------------------------------------+

+---------------------------------------+
|      Multinomial NB Tfidf(valid)      |
+---------------------------------------+
| Accuracy|    F1   |  Recall |Precision|
+---------------------------------------+
|92.367601|92.95977|89.364641|96.856287|
+---------------------------------------+

+---------------------------------------+
|       Multinomial NB Tfidf(test)      |
+---------------------------------------+
| Accuracy|    F1   |  Recall |Precision|
+---------------------------------------+
|83.583268|87.270502|77.5|99.859944|
+---------------------------------------+



### Support Vector Classifier

In [76]:
# Parameter grid

def objective(trial):
    kernel = trial.suggest_categorical('kernel', ['linear', 'poly', 'rbf', 'sigmoid'])
    if kernel is 'poly':
        degree = trial.suggest_int('degree', 1, 4)
        gamma = trial.suggest_categorical('gamma', ['scale','auto'])
        clf = SVC(kernel=kernel, degree=degree, gamma=gamma)

    if kernel in ['rbf', 'sigmoid']:
        gamma = trial.suggest_categorical('gamma', ['scale','auto'])
        clf = SVC(kernel=kernel, gamma=gamma)

    clf = SVC(kernel=kernel)

    return cross_val_score(clf, tfidf_train, y_train, cv=10, n_jobs=-1).mean()

In [77]:
# Support Vector Machines fitting

study = optuna.create_study(study_name='tfidf_svc', direction='maximize')
study.optimize(objective, n_trials=10)

[I 2021-05-10 02:37:56,480] A new study created in memory with name: tfidf_svc
/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning:

A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.

[I 2021-05-10 02:40:31,925] Trial 0 finished with value: 0.58330078125 and parameters: {'kernel': 'poly', 'degree': 3, 'gamma': 'auto'}. Best is trial 0 with value: 0.58330078125.
[I 2021-05-10 02:43:09,134] Trial 1 finished with value: 0.58330078125 and parameters: {'kernel': 'poly', 'degree': 3, 'gamma': 'scale'}. Best is trial 0 with value: 0.58330078125.
[I 2021-05-10 02:45:46,395] Trial 2 finished with value: 0.58330078125 and parameters: {'kernel': 'poly', 'degree': 3, 'gamma': 'scale'}. Best is trial 0 with value: 0.58330078125.
[I 2021-05-10 02:47:13,908] Trial 3 finished with value: 0.60615234375 and parameters: {'kernel': 'linear'}. Best is trial 3 with value: 0.60615234

In [78]:
# Best params

svc_trial = study.best_trial

svc_best = SVC(**svc_trial.params)
svc_best.fit(tfidf_train, y_train)
(svc_best.predict(tfidf_train) == y_train).mean(), (svc_best.predict(tfidf_valid) == y_valid).mean()

(0.96396484375, 0.6105919003115264)

In [79]:
# svc evaluation

evaluate_on_data(SVC(**svc_trial.params, probability=True), tfidf_train, y_train, "SVC Count")
evaluate_on_data(SVC(**svc_trial.params, probability=True), tfidf_valid, y_valid, "SVC Count", "valid")
evaluate_on_data(SVC(**svc_trial.params, probability=True), tfidf_test, y_test, "SVC Count", "test")

+---------------------------------------+
|            SVC Count(train)           |
+---------------------------------------+
| Accuracy|    F1   |  Recall |Precision|
+---------------------------------------+
|96.396484|96.864111|94.746467|99.078581|
+---------------------------------------+

+---------------------------------------+
|            SVC Count(valid)           |
+---------------------------------------+
| Accuracy|    F1   |  Recall |Precision|
+---------------------------------------+
|99.454829|99.47644|99.402093|99.550898|
+---------------------------------------+

+---------------------------------------+
|            SVC Count(test)            |
+---------------------------------------+
| Accuracy|    F1   |  Recall |Precision|
+---------------------------------------+
|99.52644|99.58159|99.166667|100.0|
+---------------------------------------+



### Random Forest

In [80]:
# Parameters grid

def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 100, 1000, 100)
    criterion = trial.suggest_categorical('criterion', ['gini', 'entropy'])
    max_depth = trial.suggest_categorical('max_depth', [4, 5, 6, None])
    max_features = trial.suggest_categorical('max_features', ['auto', 'sqrt', 'log2', None])

    clf = RandomForestClassifier(n_estimators=n_estimators, criterion=criterion, max_depth=max_depth, max_features=max_features)

    return cross_val_score(clf, tfidf_train, y_train, cv=10, n_jobs=-1).mean()

In [82]:
# Random Forest fitting

study = optuna.create_study(study_name='tfidf_rf', direction='maximize')
study.optimize(objective, n_trials=10)

[I 2021-05-10 04:15:57,295] A new study created in memory with name: tfidf_rf
[I 2021-05-10 04:26:32,537] Trial 0 finished with value: 0.6140625 and parameters: {'n_estimators': 500, 'criterion': 'entropy', 'max_depth': None, 'max_features': 'log2'}. Best is trial 0 with value: 0.6140625.
[I 2021-05-10 04:27:27,616] Trial 1 finished with value: 0.56171875 and parameters: {'n_estimators': 1000, 'criterion': 'gini', 'max_depth': 5, 'max_features': 'sqrt'}. Best is trial 0 with value: 0.6140625.
[I 2021-05-10 04:32:53,649] Trial 2 finished with value: 0.61181640625 and parameters: {'n_estimators': 200, 'criterion': 'entropy', 'max_depth': None, 'max_features': 'auto'}. Best is trial 0 with value: 0.6140625.
[I 2021-05-10 04:33:21,878] Trial 3 finished with value: 0.56171875 and parameters: {'n_estimators': 1000, 'criterion': 'gini', 'max_depth': 5, 'max_features': 'log2'}. Best is trial 0 with value: 0.6140625.
[I 2021-05-10 04:34:07,340] Trial 4 finished with value: 0.56171875 and parame

In [83]:
# Best params

rf_trial = study.best_trial

rf_best = RandomForestClassifier(**rf_trial.params)
rf_best.fit(tfidf_train, y_train)
(rf_best.predict(tfidf_train) == y_train).mean(), (rf_best.predict(tfidf_valid) == y_valid).mean()

(0.999609375, 0.5973520249221184)

In [84]:
# rf evaluation

evaluate_on_data(RandomForestClassifier(**rf_trial.params), tfidf_train, y_train, "Random Forest Count")
evaluate_on_data(RandomForestClassifier(**rf_trial.params), tfidf_valid, y_valid, "Random Forest Count", "valid")
evaluate_on_data(RandomForestClassifier(**rf_trial.params), tfidf_test, y_test, "Random Forest Count", "test")

+---------------------------------------+
|       Random Forest Count(train)      |
+---------------------------------------+
| Accuracy|    F1   |  Recall |Precision|
+---------------------------------------+
|99.960938|99.965229|99.965229|99.965229|
+---------------------------------------+

+---------------------------------------+
|       Random Forest Count(valid)      |
+---------------------------------------+
| Accuracy|    F1   |  Recall |Precision|
+---------------------------------------+
|100.0|100.0|100.0|100.0|
+---------------------------------------+

+---------------------------------------+
|       Random Forest Count(test)       |
+---------------------------------------+
| Accuracy|    F1   |  Recall |Precision|
+---------------------------------------+
|100.0|100.0|100.0|100.0|
+---------------------------------------+



### Stochastic Gradient Descent

In [85]:
# Parameters grid

def objective(trial):
    loss = trial.suggest_categorical('loss', ['log', 'modified_huber', 'perceptron'])
    penalty = trial.suggest_categorical('penalty', ['l2', 'l1', 'elasticnet'])
    max_iter = trial.suggest_int('max_iter', 1000, 2000, 100)
    learning_rate = trial.suggest_categorical('learning_rate', ['constant', 'optimal', 'invscaling', 'adaptive'])
    if learning_rate is not 'optimal':
        eta0 = trial.suggest_loguniform('eta0', 10e-6, 1)
    else:
        eta0 = 0

    clf = SGDClassifier(loss=loss, penalty=penalty, learning_rate=learning_rate, eta0=eta0, max_iter=max_iter)

    return cross_val_score(clf, count_train, y_train, cv=10, n_jobs=-1).mean()

In [86]:
# SGD fitting

study = optuna.create_study(study_name='tfidf_sgd', direction='maximize')
study.optimize(objective, n_trials=100)

[I 2021-05-10 05:57:29,623] A new study created in memory with name: tfidf_sgd
[I 2021-05-10 05:57:30,451] Trial 0 finished with value: 0.548046875 and parameters: {'loss': 'perceptron', 'penalty': 'l1', 'max_iter': 1700, 'learning_rate': 'optimal'}. Best is trial 0 with value: 0.548046875.
[I 2021-05-10 05:57:32,889] Trial 1 finished with value: 0.58662109375 and parameters: {'loss': 'modified_huber', 'penalty': 'l1', 'max_iter': 1800, 'learning_rate': 'constant', 'eta0': 0.000812443266322384}. Best is trial 1 with value: 0.58662109375.
[I 2021-05-10 05:57:33,356] Trial 2 finished with value: 0.557421875 and parameters: {'loss': 'modified_huber', 'penalty': 'l2', 'max_iter': 1100, 'learning_rate': 'optimal'}. Best is trial 1 with value: 0.58662109375.
[I 2021-05-10 05:57:33,655] Trial 3 finished with value: 0.5787109375 and parameters: {'loss': 'modified_huber', 'penalty': 'l1', 'max_iter': 1300, 'learning_rate': 'invscaling', 'eta0': 0.00019597762348780065}. Best is trial 1 with valu

In [87]:
# sgd evaluation

evaluate_on_data(SGDClassifier(**sgd_trial.params), tfidf_train, y_train, "SGD Count")
evaluate_on_data(SGDClassifier(**sgd_trial.params), tfidf_valid, y_valid, "SGD Count", "valid")
evaluate_on_data(SGDClassifier(**sgd_trial.params), tfidf_test, y_test, "SGD Count", "test")

+---------------------------------------+
|            SGD Count(train)           |
+---------------------------------------+
| Accuracy|    F1   |  Recall |Precision|
+---------------------------------------+
|58.007812|72.569533|57.315599|98.887344|
+---------------------------------------+

+---------------------------------------+
|            SGD Count(valid)           |
+---------------------------------------+
| Accuracy|    F1   |  Recall |Precision|
+---------------------------------------+
|82.320872|84.31237|78.30552|91.317365|
+---------------------------------------+

+---------------------------------------+
|            SGD Count(test)            |
+---------------------------------------+
| Accuracy|    F1   |  Recall |Precision|
+---------------------------------------+
|63.456985|75.489677|60.680851|99.859944|
+---------------------------------------+



## Creating Tfidf Vectorizer piplines

In [88]:
# logreg pipeline

logreg_pipeline = Pipeline([
     ('tfidf_vec', TfidfVectorizer()),
     ('logreg', LogisticRegression(**logreg_trial.params))
])

In [89]:
# nb pipeline

nb_pipeline = Pipeline([
      ('tfidf_vec', TfidfVectorizer()),
      ('nb', MultinomialNB(**nb_trial.params))
])

In [90]:
# svc pipeline

svc_pipeline = Pipeline([
      ('tfidf_vec', TfidfVectorizer()),
      ('svc', SVC(**svc_trial.params, probability=True))
])

In [91]:
# rf pipeline

rf_pipeline = Pipeline([
      ('tfidf_vec', TfidfVectorizer()),
      ('rf', RandomForestClassifier(**rf_trial.params))
])

In [92]:
# sgd pipeline

sgd_pipeline = Pipeline([
      ('tfidf_vec', TfidfVectorizer()),
      ('sgd', SGDClassifier(**sgd_trial.params))
])

In [93]:
# Ensemble of the above classifiers

list_clfs = [('logreg_pipeline', logreg_pipeline), ('nb_pipeline', nb_pipeline), ('svc_pipeline', svc_pipeline), ('rf_pipeline', rf_pipeline), ('sgd_pipeline', sgd_pipeline)]

tfidf_detector = VotingClassifier(estimators=list_clfs, voting='soft', n_jobs=-1)
tfidf_detector.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning:

A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.



VotingClassifier(estimators=[('logreg_pipeline',
                              Pipeline(memory=None,
                                       steps=[('tfidf_vec',
                                               TfidfVectorizer(analyzer='word',
                                                               binary=False,
                                                               decode_error='strict',
                                                               dtype=<class 'numpy.float64'>,
                                                               encoding='utf-8',
                                                               input='content',
                                                               lowercase=True,
                                                               max_df=1.0,
                                                               max_features=None,
                                                               min_df=1,
                                

In [94]:
# Prediction from voting classifier

(tfidf_detector.predict(X_valid) == y_valid).mean()
evaluate_on_data(tfidf_detector, X_train, y_train, "Ensemble Classifier Tfidf")
evaluate_on_data(tfidf_detector, X_valid, y_valid, "Ensemble Classifier Tfidf", "valid")
evaluate_on_data(tfidf_detector, X_test, y_test, "Ensemble Classifier Tfidf", "test")

/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning:

A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.



+---------------------------------------+
|    Ensemble Classifier Tfidf(train)   |
+---------------------------------------+
| Accuracy|    F1   |  Recall |Precision|
+---------------------------------------+
|94.472656|95.284905|91.474728|99.426287|
+---------------------------------------+

+---------------------------------------+
|    Ensemble Classifier Tfidf(valid)   |
+---------------------------------------+
| Accuracy|    F1   |  Recall |Precision|
+---------------------------------------+
|99.221184|99.251497|99.251497|99.251497|
+---------------------------------------+

+---------------------------------------+
|    Ensemble Classifier Tfidf(test)    |
+---------------------------------------+
| Accuracy|    F1   |  Recall |Precision|
+---------------------------------------+
|98.342541|98.550725|97.142857|100.0|
+---------------------------------------+

